In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
import pandas as pd
import glob
import os
import shutil
import sh
import subprocess
import getpass

In [2]:
# Download Location Path
mypath_download = "/ADNI_Tools/DL"
# Username of ADNI
email_key = "xxx"
# Password of ADNI
pw_key = "xxx"
# Create Download Location
mypath = mypath_download + "/adni_table"
if not os.path.exists(mypath):
    os.makedirs(mypath)
    print("Temporary directory created in " + mypath)

In [16]:
def clean_tr(path, bar): 
    now = time.time()
    files = [os.path.join(path, filename) for filename in os.listdir(path)]
    for filename in files:
        if (now - os.stat(filename).st_mtime) < bar:
            if os.path.isdir(files[0]): shutil.rmtree(filename)
            else: os.remove(filename)
                
def delet_file(path, filename): 
    file = path + filename
    if os.path.isdir(file): shutil.rmtree(file)
    else: os.remove(file)
            
def cancel_dl(): 
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(1)
    driver.execute_script("document.querySelector('downloads-manager').shadowRoot.querySelector('#downloadsList downloads-item').shadowRoot.querySelector('cr-button[focus-type=\x22cancel\x22]').click()")
    time.sleep(1)
    driver.switch_to.window(driver.window_handles[0])

def check_bl(path, bar):
    bar = 0
    while glob.glob(os.path.join(path, '*')) == []:
        bar = bar + 1
        time.sleep(2)
        if bar > bar: 
            return True

def wait_dl(path, suffix, bar):
    for i in range(bar):
        time.sleep(1)
        try: 
            if glob.glob(os.path.join(path, '*'))[0].split(".")[-1] != suffix:
                return False
        except:
            time.sleep(1)
    return True

In [18]:
print("Loading page ...")
chrome_options = webdriver.ChromeOptions() 
prefs = {'download.default_directory' : mypath}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.get("https://ida.loni.usc.edu/login.jsp")
print("Webpage opened.")
time.sleep(6)
driver.find_element(By.CLASS_NAME, "ida-cookie-policy-accept").click()
print("Cookie accepted.")
time.sleep(6)
driver.find_element(By.CLASS_NAME, "ida-user-menu-icon").click()
time.sleep(2)
email = driver.find_element(By.NAME, "userEmail")
pw = driver.find_element(By.NAME, "userPassword")
email.send_keys(email_key)
pw.send_keys(pw_key)
pw.send_keys(Keys.RETURN)
print("Login information entered.")
time.sleep(2)
driver.find_element(By.CLASS_NAME, "download").click()
time.sleep(2)
driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div[2]/div[2]/div/div[4]/div[2]/div/div[1]/span").click()
driver.find_element(By.ID, "ygtvlabelel48").click()
print("Directed to data downloading page.")
info_list = [];
info_list = driver.find_elements(By.XPATH, "//td[@class='contentFont']")
link_list = [];
link_list = driver.find_elements(By.XPATH, "//td[@class='contentFont']/a")
driver.execute_script('''window.open("http://google.com","_blank");''')
driver.switch_to.window(driver.window_handles[1])
driver.get('chrome://downloads')
driver.switch_to.window(driver.window_handles[0])
print("Loaded download page.")
file_label = []
file_name =[]
file_vars = []
file_version = []
lar_file = []
err_file = []
time.sleep(3)

for i in range(0, int(len(link_list)), 1):
#for i in range(84, 86, 1):
    # Click each download links    
    l_name = link_list[i].text.replace('\n', '').strip()
    version = info_list[2*i+1].text.replace('  Version: ', '')
    file_label.append(l_name)
    file_version.append(version)
    if "PDF" in l_name or "Methods" in l_name or "Documentation" in l_name or "Document" in l_name or "METHODS" in l_name or "About" in l_name:
        file_name.append('')
        #file_vars.append('')
        file_vars.append('Methods file.')
        time.sleep(1)
        print("Skip methods file.")
        continue
    try:
        link_list[i].click()
        time.sleep(1)
        driver.find_element(By.CLASS_NAME, "preparing-download-close-btn").click()
    except:
        pass
    print("(" + str(i) + "/" + str(int(len(link_list))) + ")" + "Downloading " + l_name + " ... ")
    # Check bad links
    time.sleep(2)
    if check_bl(mypath, 3):
        file_name.append('')
        file_vars.append('')
        err_file.append(l_name)
        print('Bad link')
        continue
    # Check pop-ups
    time.sleep(3)
    if len(driver.window_handles) > 2: 
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[2])
        time.sleep(1)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        file_name.append('')
        file_vars.append('webpage_pdf')
        lar_file.append(l_name)
        print('Skip discription file ...')
        continue
    # Wait for downloading
    timeout = wait_dl(mypath, "crdownload", 60)
    # large file
    if timeout: 
        name = glob.glob(os.path.join(mypath, '*'))
        cancel_dl()
        head, tail = os.path.split(name[0])
        file_name.append(tail.replace('.crdownload', ''))
        file_vars.append('large')
        lar_file.append(l_name)
        print("Skip the large file ...")
        continue
    # classify downloaded file type and add values
    time.sleep(2)
    name = glob.glob(os.path.join(mypath, '*'))
    head, tail = os.path.split(name[0])
    file_name.append(tail)
    print("Added file name " + tail)
    # directory
    if len(name[0].split(".")) < 2 or os.path.isdir(name[0]):
        file_vars.append('non-attributed/large')
        err_file.append(l_name)
        shutil.rmtree(name[0])
        time.sleep(1)
        print("Removed non-attributed/large file.")
        continue
    # non-csv file
    if tail.split(".")[-1] != "csv":
        file_vars.append('non-csv')
        err_file.append(l_name)
        os.remove(name[0])
        time.sleep(1)
        print("Removed non-csv file.")
        continue
    # csv file    
    try:
        file_vars.append(pd.read_csv(name[0]).columns.values.tolist())
        print("Variables added")
    except:
        file_vars.append('not avaliable')
        print("Variables not avaliable")
    os.remove(name[0])
    time.sleep(1) 
    print("Removed file.") 


Loading page ...
Webpage opened.
Cookie accepted.
Login information entered.
Directed to data downloading page.
Loaded download page.
(0/994)Downloading Diagnosis and Symptoms Checklist [ADNI1,GO,2] ... 
Added file name ADSXLIST.csv
Variables added
Removed file.
(1/994)Downloading Diagnostic Summary - Baseline Changes [ADNI1,GO,2,3] ... 
Added file name DXSUM_PDXCONV_ADNIALL.csv
Variables added
Removed file.
(2/994)Downloading Diagnostic Summary [ADNI1,GO,2,3] ... 
Added file name DXSUM_PDXCONV_ADNIALL.csv
Variables added
Removed file.
(3/994)Downloading ADAS Sub-Scores and Total Scores [ADNI1] ... 
Added file name ADASSCORES.csv
Variables added
Removed file.
(4/994)Downloading ADSP Phenotype Harmonization Consortium (PHC) - Composite Cognitive Scores Dictionary [ADNI1,GO,2,3] ... 
Added file name ADSP_PHC_COGN_DICT_10_05_22.csv
Variables added
Removed file.
Skip methods file.
(6/994)Downloading ADSP Phenotype Harmonization Consortium (PHC) - Composite Cognitive Scores [ADNI1,GO,2,3] .

Added file name ADMCADNI1SCPATIENTDRUGCLASSES.csv
Variables added
Removed file.
(67/994)Downloading ADMC Duke ADNI2/GO Drug Classes ... 
Added file name BLCHANGE.csv
Variables added
Removed file.
(68/994)Downloading ADMC Duke Biocrates P180 Kit Flow injection analysis Dictionary [ADNI1] ... 
Added file name ADMCDUKEP180FIA_DICT.csv
Variables added
Removed file.
(69/994)Downloading ADMC Duke Biocrates p180 Kit Flow injection analysis Dictionary [ADNIGO,2] ... 
Added file name ADMCDUKEP180FIAADNI2GO_DICT.csv
Variables added
Removed file.
(70/994)Downloading ADMC Duke Biocrates P180 Kit Flow injection analysis [ADNI1] ... 
Added file name ADMCDUKEP180FIA_01_15_16.csv
Variables added
Removed file.
(71/994)Downloading ADMC Duke Biocrates p180 Kit Flow injection analysis [ADNIGO,2] ... 
Added file name ADMCDUKEP180FIAADNI2GO.csv
Variables added
Removed file.
(72/994)Downloading ADMC Duke Biocrates P180 Kit Ultra Performance Liquid Chromatography Dictionary ... 
Added file name ADMCDUKEP180UP

/Users/xinyuguo/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (90,101,108) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Variables added
Removed file.
Skip methods file.
(107/994)Downloading ADMC Nightingale Platform NMR Analysis of Lipoproteins and Metabolites Dictionary [ADNI1,GO,2] ... 
Added file name ADNINIGHTINGALE_DICT_20210219.csv
Variables added
Removed file.
Skip methods file.
(109/994)Downloading ADMC Nightingale Platform NMR Analysis of Lipoproteins and Metabolites [ADNI1,GO,2] ... 
Added file name ADNINIGHTINGALE_20210219.csv
Variables added
Removed file.
(110/994)Downloading ADMC Nightingale Platform NMR Post-Unblinding Re-Analysis of Lipoproteins and Metabolites Dictionary [ADNI1,GO,2] ... 
Added file name ADNINIGHTINGALE2_DICT.csv
Variables added
Removed file.
Skip methods file.
(112/994)Downloading ADMC Nightingale Platform NMR Post-Unblinding Re-Analysis of Lipoproteins and Metabolites [ADNI1,GO,2] ... 
Added file name ADNINIGHTINGALE2.csv
Variables added
Removed file.
(113/994)Downloading ADMC Phenomenome Phosphatidylcholine, Lysophosphatidylcholine Flow Injection Negative Ionization L

/Users/xinyuguo/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3,11,16,32,33,38,52,66,67,72,84,87) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

Added file name MAYOADIRL_MRI_FMRI_NFQ_05_04_20.csv
Variables added
Removed file.
(702/994)Downloading Mayo (Jack Lab) - Network Failure Quotient for rs-fMRI [ADNIGO,2,3] ... 
Added file name MAYOADIRL_MRI_FMRI_NFQ_09_07_22.csv
Variables added
Removed file.
(703/994)Downloading Mayo (Jack Lab) - Network Failure Quotient for rs-fMRI [ADNIGO,2,3] ... 
Added file name MAYOADIRL_MRI_FMRI_NFQ_05_09_22.csv
Variables added
Removed file.
(704/994)Downloading Mayo (Jack Lab) - Network Failure Quotient for rs-fMRI [ADNIGO,2,3] ... 
Added file name MAYOADIRL_MRI_FMRI_NFQ_08_14_20.csv
Variables added
Removed file.
(705/994)Downloading Mayo (Jack Lab) - Network Failure Quotient for rs-fMRI [ADNIGO,2,3] ... 
Added file name MAYOADIRL_MRI_FMRI_NFQ_01_11_21.csv
Variables added
Removed file.
(706/994)Downloading Mayo (Jack Lab) - Network Failure Quotient for rs-fMRI [ADNIGO,2,3] ... 
Added file name MAYOADIRL_MRI_FMRI_NFQ_11_26_19.csv
Variables added
Removed file.
(707/994)Downloading Mayo (Jack Lab) -

Added file name UCBERKELEYAV1451_01_14_21.csv
Variables added
Removed file.
(763/994)Downloading UC Berkeley - AV1451 analysis [ADNI2] ... 
Added file name UCBERKELEYAV1451_06_15_16.csv
Variables added
Removed file.
(764/994)Downloading UC Berkeley - AV1451 Analysis [ADNI2] ... 
Added file name UCBERKELEYAV1451_10_17_16.csv
Variables added
Removed file.
(765/994)Downloading UC Berkeley - AV1451 Analysis [ADNI2] ... 
Added file name UCBERKELEYAV1451.csv
Variables added
Removed file.
(766/994)Downloading UC Berkeley - AV1451 Partial Volume Corrected Analysis [ADNI2,3] ... 
Added file name UCBERKELEYAV1451_PVC_02_04_20.csv
Variables added
Removed file.
(767/994)Downloading UC Berkeley - AV1451 Partial Volume Corrected Analysis [ADNI2,3] ... 
Added file name UCBERKELEYAV1451_PVC_05_12_20.csv
Variables added
Removed file.
(768/994)Downloading UC Berkeley - AV1451 PVC Analysis [ADNI2,3] ... 
Added file name UCBERKELEYAV1451_PVC_01_12_22.csv
Variables added
Removed file.
(769/994)Downloading 

Removed file.
(826/994)Downloading UC Berkeley - FDG Analysis [ADNI1,GO,2,3] ... 
Added file name UCBERKELEYFDG_02_04_20.csv
Variables added
Removed file.
(827/994)Downloading UC Berkeley - FDG Analysis [ADNI1,GO,2,3] ... 
Added file name UCBERKELEYFDG_08_27_19.csv
Variables added
Removed file.
(828/994)Downloading UC Berkeley - FDG Analysis [ADNI1,GO,2,3] ... 
Added file name UCBERKELEYFDG_09_05_18.csv
Variables added
Removed file.
(829/994)Downloading UC Berkeley - FDG Analysis [ADNI1,GO,2,3] ... 
Added file name UCBERKELEYFDG_03_02_22.csv
Variables added
Removed file.
(830/994)Downloading UC Berkeley - FDG Analysis [ADNI1,GO,2] ... 
Added file name UCBERKELEYFDG_07_30_15.csv
Variables added
Removed file.
(831/994)Downloading UC Berkeley - FDG Analysis [ADNI1,GO,2] ... 
Added file name UCBERKELEYFDG_03_13_14.csv
Variables added
Removed file.
(832/994)Downloading UC Berkeley - FDG Analysis [ADNI1,GO,2] ... 
Added file name UCBERKELEYFDG_11_17_13.csv
Variables added
Removed file.
Skip 

Added file name UCSFFSX51_08_01_14.csv
Variables added
Removed file.
(881/994)Downloading UCSF - Cross-Sectional FreeSurfer (5.1) [ADNI1,GO,2] Dictionary ... 
Added file name UCSFFSX51_DICT_10_31_13.csv
Variables added
Removed file.
(882/994)Downloading UCSF - Cross-Sectional FreeSurfer (6.0) [ADNI3] ... 
Added file name UCSFFSX6_08_27_19.csv
Variables added
Removed file.
(883/994)Downloading UCSF - Cross-Sectional FreeSurfer (6.0) [ADNI3] ... 
Added file name UCSFFSX6_11_08_19.csv
Variables added
Removed file.
(884/994)Downloading UCSF - Cross-Sectional FreeSurfer (6.0) [ADNI3] ... 
Added file name UCSFFSX6_02_05_20.csv
Variables added
Removed file.
(885/994)Downloading UCSF - Cross-Sectional FreeSurfer (6.0) [ADNI3] ... 
Added file name UCSFFSX6_03_02_21.csv
Variables added
Removed file.
(886/994)Downloading UCSF - Cross-Sectional FreeSurfer (6.0) [ADNI3] ... 
Added file name UCSFFSX6_06_07_22.csv
Variables added
Removed file.
(887/994)Downloading UCSF - Cross-Sectional FreeSurfer (F

Added file name UPENNSPARE_-_MCI_-_DICT.csv
Variables added
Removed file.
Skip methods file.
(943/994)Downloading UPENN - Spatial Pattern of Abnormalities for Recognition of Early MCI to AD conversion (SPARE-MCI) [ADNI1] ... 
Added file name UPENNSPARE_-_MCI.csv
Variables added
Removed file.
(944/994)Downloading UPENN - Third Batch Analysis of CSF Biomarkers Dictionary [ADNIGO,2] ... 
Added file name UPENNBIOMK7_DICT.csv
Variables added
Removed file.
(945/994)Downloading UPENN - Third Batch Analysis of CSF Biomarkers [ADNIGO,2] ... 
Added file name UPENNBIOMK7.csv
Variables added
Removed file.
(946/994)Downloading UPENN CSF Biomarkers Elecsys Dictionary [ADNI1,GO,2] ... 
Added file name UPENNBIOMK9_DICT.csv
Variables added
Removed file.
(947/994)Downloading UPENN CSF Biomarkers Elecsys [ADNI1,GO,2] ... 
Added file name UPENNBIOMK9.csv
Variables added
Removed file.
(948/994)Downloading USC - DTI ROI Summary Measures Dictionary [ADNIGO,2] ... 
Added file name ADNI_DTIROI_DICT_10_03_19.cs

In [50]:
try:
    # Output
    file_name_map = pd.DataFrame({"file_label": file_label, 
                                  "file_name": file_name, 
                                  "file_version": file_version, 
                                  "vars": file_vars_tem})
    print("File merge successfully.")
    shutil.rmtree(mypath, ignore_errors=True)
    print("Temporary directory deleted.")
    file_name_map.to_csv(mypath_download+"/ADNI_map.csv")
    print("ADNI_map.csv created in " + mypath_download + "/ADNI_map.csv")
except:
    print("Fail to create the csv file")

# Quit Driver
driver.quit()

Merge successfully,
Temporary directory deleted
ADNI_map.csv created in /Users/xinyuguo/Desktop/JHU/AD Computing Group/ADNI_Tools/DL/ADNI_map.csv
